# Biharmonic equation and stresses in polar coordinates

## Background

We had already derived the biharmonic equation in rectangular Cartesian coordinate system. So we know:

\begin{gather}
\nabla^4 \phi = 0, \\
\text{or,} \quad \left( \frac{\partial^4}{\partial x^4} + 2 \frac{\partial^4}{\partial x^2 \partial y^2} + \frac{\partial^4}{\partial y^4} \right) \phi = 0
\end{gather}

Now, we wish to obtain the version in the polar coordinate system, i.e. the $r, \theta$ coordinate system. The whole derivation is, in essence, a transformation from the rectangular Cartesian coordinate system to the polar coordinate system. We follow the steps outlined in the classic "Theory of Elasticity" 3rd edition, by Timoshenko and Goodier. 

We begin by noting that $x = r \cos\theta$ and $y = r\sin \theta$. Then, we obtain

\begin{align}
\frac{\partial r}{\partial x} &= \cos \theta \\
\frac{\partial r}{\partial y} &= \sin \theta \\
\frac{\partial \theta}{\partial x} & = - \frac{\sin \theta}{r} \\
\frac{\partial \theta}{\partial y} & = \frac{\cos \theta}{r} \\
\end{align}

For a generic function $f(r,\theta)$, we have


\begin{align}
\frac{\partial f}{\partial x} &= \frac{\partial f}{\partial r}\frac{\partial r}{\partial x} + \frac{\partial f}{\partial \theta}\frac{\partial \theta}{\partial x} \\
\frac{\partial f}{\partial y} &= \frac{\partial f}{\partial r}\frac{\partial r}{\partial y} + \frac{\partial f}{\partial \theta}\frac{\partial \theta}{\partial y} 
\end{align}

Up to this point, we could easily proceed "by hand". However, our final aim is to obtain the polar coordinate versions of $\displaystyle \frac{\partial^4 f}{\partial x^4}$, $\displaystyle \frac{\partial^4 f}{\partial x^2\partial y^2}$, and $\displaystyle \frac{\partial^4 f}{\partial y^4}$. Doing {\em these} by hand involves a lot of time and effort. 

However, we can take advantage of the power of SymPy to automate these derivations. 

## Definitions in SymPy
We begin by invoking SymPy and defining the symbols $r$ and $\theta$.

In [1]:
import sympy as sp

r, theta = sp.symbols('r, theta')

We define a generic function $f(r,\theta)$ as:

In [2]:
f = sp.Function('f')(r,theta)

We then write the definitions as in Eqs (3)-(6):

In [3]:
def delr_delx():
    return sp.cos(theta)
    
def delr_dely():
    return sp.sin(theta)

def deltheta_delx():
    return -sp.sin(theta)/r

def deltheta_dely():
    return sp.cos(theta)/r

And, then we write the definitions as in Eqs (7) and (8):

In [4]:
def del_delx(f):
    return delr_delx()*sp.diff(f,r,1) + deltheta_delx()*sp.diff(f,theta,1)

def del_dely(f):
    return delr_dely()*sp.diff(f,r,1) + deltheta_dely()*sp.diff(f,theta,1)

Up to this point, we have merely translated the equations which we had obtained by hand into the language of SymPy. However, we are now at the point where we need to find the second order derivatives. These we shall obtain not by hand, but by using the power of SymPy by making the definitions of the first order derivatives operate on themselves. Thus:

In [5]:
def del2_delx2(f):
    return del_delx(del_delx(f))

def del2_dely2(f):
    return del_dely(del_dely(f))

### Definition of the polar biharmonic

We finally wish to go to the biharmonic equation. But for that we need not explicitly find the third order derivatives and then the fourth order derivatives. Instead, we can define the polar version of the Laplacian operator $\displaystyle \frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2}$ as follows:

In [6]:
def polarLaplacian(f):
    return (del2_delx2(f)+del2_dely2(f))

That's the definition. But, let's check what the polar Laplacian looks:

In [7]:
polarLaplacian(f)

(sin(theta)*Derivative(f(r, theta), (r, 2)) + cos(theta)*Derivative(f(r, theta), r, theta)/r - cos(theta)*Derivative(f(r, theta), theta)/r**2)*sin(theta) + (cos(theta)*Derivative(f(r, theta), (r, 2)) - sin(theta)*Derivative(f(r, theta), r, theta)/r + sin(theta)*Derivative(f(r, theta), theta)/r**2)*cos(theta) - (-sin(theta)*Derivative(f(r, theta), r) + cos(theta)*Derivative(f(r, theta), r, theta) - sin(theta)*Derivative(f(r, theta), (theta, 2))/r - cos(theta)*Derivative(f(r, theta), theta)/r)*sin(theta)/r + (sin(theta)*Derivative(f(r, theta), r, theta) + cos(theta)*Derivative(f(r, theta), r) - sin(theta)*Derivative(f(r, theta), theta)/r + cos(theta)*Derivative(f(r, theta), (theta, 2))/r)*cos(theta)/r

So that looks a bit daunting. What happens if we redefine the polar Laplacian invoking the symbolic simplifications that SymPy allows:

In [8]:
def polarLaplacian(f):
    return (del2_delx2(f)+del2_dely2(f)).simplify()

In [9]:
polarLaplacian(f)

Derivative(f(r, theta), (r, 2)) + Derivative(f(r, theta), r)/r + Derivative(f(r, theta), (theta, 2))/r**2

That's much better! We can now immediately go to the polar version of the biharmonic operator by making the Laplacian operator operate on itself. Thus:

In [10]:
def polarbiharmonic(f):
    return polarLaplacian(polarLaplacian(f))

Let's check what the polar biharmonic looks like:

In [11]:
polarbiharmonic(f)

(r**4*Derivative(f(r, theta), (r, 4)) + 2*r**3*Derivative(f(r, theta), (r, 3)) - r**2*Derivative(f(r, theta), (r, 2)) + 2*r**2*Derivative(f(r, theta), (r, 2), (theta, 2)) + r*Derivative(f(r, theta), r) - 2*r*Derivative(f(r, theta), r, (theta, 2)) + 4*Derivative(f(r, theta), (theta, 2)) + Derivative(f(r, theta), (theta, 4)))/r**4

Now, imagine obtaining this by hand!

However, our work is far from over. To address a full-scale problem from 2D elasticity, we have to solve the polar biharmonic equation subject to certain boundary conditions. These boundary conditions are often expressed in terms of the stress fields. So, we need to find the relations between the stresses and the Airy stress function. 

### Definition of the stresses in polar coordinates

To find the relations between the stresses and the Airy stress function in the polar coordinate system, we proceed again by transforming from the rectangular Cartesian coordinate system. This transformation involves two steps. 

**First step:** Obtain the expressions of $\displaystyle \sigma_{xx} = \frac{\partial^2 \phi}{\partial y^2}$, $\displaystyle \sigma_{yy} = \frac{\partial^2 \phi}{\partial x^2}$, and $\displaystyle \sigma_{xy} = -\frac{\partial^2 \phi}{\partial x \partial y}$ in terms of the corresponding derivatives of the Airy stress function in polar coordinates.

**Second step:** Transform the stress matrix $\displaystyle \bar{\bar{\sigma}}_{\rm rect} = \begin{bmatrix} \sigma_{xx} & \sigma_{xy} \\ \sigma_{xy} & \sigma_{yy}\end{bmatrix}$ to $\displaystyle \bar{\bar\sigma}_{\rm pol} = \begin{bmatrix} \sigma_{rr} & \sigma_{r\theta} \\ \sigma_{r\theta} & \sigma_{\theta\theta}\end{bmatrix}$ through the rotation matrix $\bar{\bar{Q}} = \displaystyle \begin{bmatrix} \cos \theta & -\sin \theta \\ \sin \theta & \cos \theta \end{bmatrix}$ using the relation:
\begin{gather}
\bar{\bar{\sigma}}_{\rm pol} = \bar{\bar{Q}} \; \bar{\bar{\sigma}}_{\rm rect} \; \bar{\bar{Q}}^{\sf T}
\end{gather}

On to the implementation of the **First step** now:

In [12]:
def sigma_xx(f):
    return del2_dely2(f)

def sigma_yy(f):
    return del2_delx2(f)

def sigma_xy(f):
    return -del_delx(del_dely(f))

Using the above definitions, we set up $\bar{\bar{\sigma}}_{\rm rect}$:

In [13]:
def sigma_rect(f):
    return sp.Matrix([[sigma_xx(f), sigma_xy(f)],[sigma_xy(f), sigma_yy(f)]])

We next write the rotation matrix, $\bar{\bar{Q}}$:

In [14]:
Q = sp.Matrix([[sp.cos(theta), sp.sin(theta)],[-sp.sin(theta), sp.cos(theta)]])

We next implement the **Second step** as:

In [15]:
def sigma_polar(f):
    return Q*sigma_rect(f)*(Q.T)

And, finally extract the individual components $\sigma_{rr}$, $\sigma_{\theta\theta}$, and $\sigma_{r\theta}$:

In [16]:
def sigma_rr(f):
    return (sigma_polar(f)[0,0]).simplify().expand()

def sigma_tt(f):
    return (sigma_polar(f)[1,1]).simplify().expand()

def sigma_rt(f):
    return (sigma_polar(f)[0,1]).simplify().expand()

Note that in the above definitions, we have additionally invoked the simplification and expansion functionality that SymPy allows to make the expressions of the stress fields less daunting. It is important to note that it is not just a matter of aesthetics to present these expressions in the simplest possible terms. It does matter for latter operations that the expressions indeed are simple and amenable for solving problems. 

So, on to problems now ...

We will follow the development of the ideas presented in Timoshenko and Goodier. 

First, we will consider a $\theta$-independent Airy stress function, i.e. $\phi(r)$. 

In [18]:
phi = sp.Function('phi')(r)
display(phi)

phi(r)

Let's quickly check what the polar biharmonic of $\phi(r)$ looks like now that we dropped the $\theta$ dependence:

In [20]:
polarbiharmonic(phi)

Derivative(phi(r), (r, 4)) + 2*Derivative(phi(r), (r, 3))/r - Derivative(phi(r), (r, 2))/r**2 + Derivative(phi(r), r)/r**3

Now, we would like to find the solution of the biharmonic equation, $\nabla^4 \phi = 0$:

In [19]:
sp.dsolve(polarbiharmonic(phi))

Eq(phi(r), C1 + C2*r**2 + C3*r**2*log(r) + C4*log(r))

That's it! That simple one line gives us the solution to the fourth order equation. Anyone who has tried to solve the fourth order equation by hand can attest to the fact that how much labour this step is saving us. I had emphasized this point heavily during my lecture so that the students appreciate the power of SymPy.  

Anyway, now that we have the solution, we will rewrite it in the notation used by Timoshenko and Goodier. 

In [21]:
A, B, C, D = sp.symbols('A, B, C, D')
phi = A*sp.log(r) + B*r**2*sp.log(r) + C*r**2 + D
display(phi)

A*log(r) + B*r**2*log(r) + C*r**2 + D

To determine the corresponding stress fields, we use our earlier definitions. We just have to call them:

In [27]:
sigmarr = sigma_rr(phi)
sigmatt = sigma_tt(phi)
sigmart = sigma_rt(phi)
display(sigmarr, sigmatt, sigmart)

A/r**2 + 2*B*log(r) + B + 2*C

-A/r**2 + 2*B*log(r) + 3*B + 2*C

0

Note that in the above way of displaying the various expressions the actual variable name does not appear. We can get around this little issue as follows:

In [31]:
from IPython.display import Math, Latex

display(Math(r'\sigma_{{rr}} = {}'.format(sp.latex(sigmarr))))
display(Math(r'\sigma_{{\theta\theta}} = {}'.format(sp.latex(sigmatt))))
display(Math(r'\sigma_{{r\theta}} = {}'.format(sp.latex(sigmart))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

If we are investigating the stress field for a geometry where there is no hole at the origin then we must set $A=0$ and $B=0$. Otherwise $\sigma_{rr}$ and $\sigma_{\theta\theta}$ will become infinite. In such a case, we have the following:

In [33]:
sigmarr_nohole = sigmarr.subs([(A,0),(B,0)])
sigmatt_nohole = sigmatt.subs([(A,0),(B,0)])

display(Math(r'\sigma_{{rr}}^{{\rm no-hole}} = {}'.format(sp.latex(sigmarr_nohole))))
display(Math(r'\sigma_{{tt}}^{{\rm no-hole}} = {}'.format(sp.latex(sigmatt_nohole))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The above formulae show that the radial and hoop stresses correspond to a state of uniform tension or uniform compression. 

Next, if there is a hole then we can have various other solutions. Suppose we take $A \neq 0$ and $B=0$. Then, we have the following:

In [37]:
sigmarr_hole = sigmarr.subs(B,0)
sigmatt_hole = sigmatt.subs(B,0)


display(Math(r'\sigma_{{rr}}^{{\rm hole}} = {}'.format(sp.latex(sigmarr_hole))))
display(Math(r'\sigma_{{\theta\theta}}^{{\rm hole}} = {}'.format(sp.latex(sigmatt_hole))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Students who have done an undergraduate course on Strength of Materials/Mechanics of Materials/Mechanics of Solids can identify that this form of the stress fields emerges in the problem of a thick-walled pressure vessel. 

So we will now focus on the thick-walled pressure vessel problem. 

## Solution to the thick-walled pressure vessel problem

Let the inner radius be $a$ and the outer radius be $b$. Then the boundary conditions are:

\begin{align}
\sigma_{rr} &= -p_{\rm in} \quad \text{at $r=a$} \\
\sigma_{rr} &= -p_{\rm out} \quad \text{at $r=b$}
\end{align}
where $p_{\rm in}$ and $p_{out}$ are the pressures inside and outside the pressure vessel, respectively. We will need to solve for the two unknown constants $A$ and $C$ using these two boundary conditions. We can again implement this solution in SymPy. 

First, we set up the two equations from the boundary conditions as follows:

In [51]:
a,b=sp.symbols('a,b')
pin, pout = sp.symbols('p_in, p_out')

lhs = sigmarr_hole.subs(r,a)
rhs = -pin
eq1 = sp.Eq(lhs,rhs)
display(eq1)

lhs = sigmarr_hole.subs(r,b)
rhs = -pout
eq2 = sp.Eq(lhs,rhs)
display(eq2)

Eq(A/a**2 + 2*C, -p_in)

Eq(A/b**2 + 2*C, -p_out)

Next, we solve the above two equations to obtain the values of $A$ and $C$ as follows:

In [52]:
soln, = sp.linsolve([eq1,eq2],(A,C))

Avalue = soln[0]
Cvalue = soln[1]

display(Math(r'A = {}'.format(sp.latex(Avalue))))
display(Math(r'C = {}'.format(sp.latex(Cvalue))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

To obtain the solutions to the stress field, we just have to make the appropriate substitutions:

In [53]:
sigmarr_hole_soln = sigmarr_hole.subs([(A,Avalue),(C,Cvalue)])
sigmatt_hole_soln = sigmatt_hole.subs([(A,Avalue),(C,Cvalue)])
display(Math(r'\sigma_{{rr}}^{{\rm hole}} = {}'.format(sp.latex(sigmarr_hole_soln))))
display(Math(r'\sigma_{{\theta\theta}}^{{\rm hole}} = {}'.format(sp.latex(sigmatt_hole_soln))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

These solutions match with those given in Timoshenko and Goodier!